In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [2]:
model = torch.load('checkpoints/G_model.pth')


In [3]:


model = model.cuda()
model.eval()

DataParallel(
  (module): Generator(
    (start): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(6, 48, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(48, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): GELU(approximate='none')
    )
    (trane256): Sequential(
      (0): TransformerEncoder(
        (attn): DAttn(
          (qkv): Conv2d(48, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (qkv_dwconv): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=144, bias=False)
          (project_out): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (feed_forward): FeedForward(
          (norm): InstanceNorm2d(48, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (conv): Sequential(
            (0): Conv2d(48, 254, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): Conv2d(254, 254, kernel_size=(3, 3), stride=(1, 1), paddi

In [ ]:
with torch.no_grad():
    
    test_img = torch.randn(1,3,).float().cuda()
    test_mask = torch.from_numpy(ne_mask[:,:,:512,:512]).float().cuda()
    out = model.forward(inp_img.cuda(), test_mask.cuda())

In [7]:
import glob
import os
import mat73
data_path = "/tng4/users/skayasth/Yearly/2023/Jan/TCEQ/Data_for_PCNN"
File_List = sorted(glob.glob(os.path.join(
    data_path, 'Mask_Mean*.mat')))
MODIS_AOD_file = mat73.loadmat(os.path.join(
    data_path, "Regridded_MODIS_AOD_4km_Mid_2017_2022_CONUS.mat"))
PUS4k = mat73.loadmat(os.path.join(
    data_path, "Regridded_PUS_4km_2016_CONUS.mat"))

In [8]:
file= mat73.loadmat(File_List[0])

obs = file[f'Mean_PM_STNs_{2018}']
obs = np.nan_to_num(obs, nan=np.nan).astype(np.float32)
obs = np.einsum('ijk->kij', obs)
obs = obs / 450.0
obs = np.nan_to_num(obs, nan=1).astype(np.float32)
Mask = file[f"Mask_PM_STNs_{2018}"]

In [9]:
import pickle
with open("/tng4/users/skayasth/Yearly/2023/June/TCEQ_Final/Phase1/runs/May_300/maxes.pkl", 'rb') as f:
    arr_loaded = pickle.load(f)


_, modis_max, pus_max = arr_loaded

In [10]:
one_obs = obs[1]
one_obs[one_obs<0] = 0
one_mask = Mask[:,:,1]
one_modis = MODIS_AOD_file['MODIS_AOD'][:,:,10] / modis_max
one_pus = PUS4k['PUS_CONUS'] / pus_max

In [11]:
ne_data = np.array([one_obs, one_modis, one_pus])
ne_data = np.expand_dims(ne_data, axis=0)
ne_mask = np.array([one_mask, np.ones_like(one_mask), np.ones_like(one_mask)])
ne_mask = np.expand_dims(ne_mask, axis=0)


In [12]:
# Define the desired padded dimensions

def _in(original_array, constant_values=0):
    
    padded_height, padded_width = 896, 1536


    # Get the original array shape
    original_height, original_width, original_channels= original_array.shape

    # Calculate the required amount of padding
    pad_height = padded_height - original_height
    pad_width = padded_width - original_width

    # Calculate the padding sizes for each dimension
    top_padding = pad_height // 2
    bottom_padding = pad_height - top_padding
    left_padding = pad_width // 2
    right_padding = pad_width - left_padding


    # Perform the zero-padding
    padded_array = np.pad(original_array, ((top_padding, bottom_padding), (left_padding, right_padding), (0, 0)), mode='constant', constant_values=constant_values)

    unpadded_array = padded_array[top_padding:top_padding+original_height, left_padding:left_padding+original_width, :]

    return padded_array.transpose(2,0,1)

ne_data_padded = _in(ne_data[0].transpose(1,2,0)) 
ne_mask_padded = _in(ne_mask[0].transpose(1,2,0), constant_values=1)

ne_data_padded = np.expand_dims(ne_data_padded, axis=0)
ne_mask_padded = np.expand_dims(ne_mask_padded, axis=0)

In [15]:
# np.save("input_data.npy", ne_data)
# np.save("input_mask.npy", ne_mask)

: 

In [13]:
with torch.no_grad():
    
    test_img = torch.from_numpy(ne_data[:,:,:512,:512]).float().cuda()
    test_mask = torch.from_numpy(ne_mask[:,:,:512,:512]).float().cuda()
    
   
    
   
    # test_img =  torch.from_numpy(ne_data_padded).float()
    # test_mask = torch.from_numpy(ne_mask_padded).float()
    
    
    
    inp_img = test_img * test_mask
    
    out = model.forward(inp_img.cuda(), test_mask.cuda())
    out_22 = out  * (1 - test_mask.cuda()) + inp_img.cuda()

ic| x.shape: torch.Size([1, 48, 512, 512])


NameError: name 'exit' is not defined

In [ ]:
def split_image(image_array, size=(512, 512), overlap=0.50):
    channels, width, height = image_array.shape
    stride_width = int(size[0] * (1 - overlap))
    stride_height = int(size[1] * (1 - overlap))

    cropped_images = []
    positions = []

    for left in range(0, width - size[0] + 1, stride_width):
        for top in range(0, height - size[1] + 1, stride_height):
            right = left + size[0]
            bottom = top + size[1]
            cropped_array = image_array[:, left:right, top:bottom]
            cropped_images.append(cropped_array)
            positions.append((left, right, top, bottom))
            
    cropped_array = image_array[:, width-512:, :512]
    cropped_images.append(cropped_array)
    positions.append((width-512, width, 0, 512))
    cropped_array = image_array[:, width-512:, height-512:]
    cropped_images.append(cropped_array)
    positions.append((width-512, width, height-512, height))

    return np.array(cropped_images), positions

In [ ]:
cropped_images,pos = split_image(ne_data[0], size=(512, 512), overlap=0.20)
cropped_masks,posm = split_image(ne_mask[0], size=(512, 512), overlap=0.20)

In [ ]:
cropped_images.shape

In [ ]:
with torch.no_grad():
    outs = []
    for img, mask in zip(cropped_images,cropped_masks):
        
        img = np.expand_dims(img, axis=0)
        mask = np.expand_dims(mask, axis=0)
        
        test_img = torch.from_numpy(img).float().cuda()
        test_mask = torch.from_numpy(mask).float().cuda()
    
   
    
   
    # # test_img =  torch.from_numpy(ne_data_padded).float()
    # # test_mask = torch.from_numpy(ne_mask_padded).float()
    
    
    
        inp_img = test_img * test_mask
    
        out = model.forward(inp_img.cuda(), test_mask.cuda())
        
        out_22 = out  * (1 - test_mask.cuda()) + inp_img.cuda()
        
        outs.append(out_22[0].cpu().detach().numpy())

In [ ]:
num_images = cropped_images.shape[0]
num_cols = int(np.sqrt(num_images))
num_rows = int(np.ceil(num_images / num_cols))

fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 10))

for i, ax in enumerate(axes.flat):
    if i < num_images:
        ax.imshow(cropped_images[i].transpose(1, 2, 0)[:, :, -1])
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
num_images = cropped_images.shape[0]
num_cols = int(np.sqrt(num_images))
num_rows = int(np.ceil(num_images / num_cols))

fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 10))

for i, ax in enumerate(axes.flat):
    if i < num_images:
        ax.imshow(outs[i].transpose(1, 2, 0)[:, :, 0])
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# outs
main_out = np.zeros((3, 768, 1456))

In [ ]:
pos

In [ ]:
for i, var in enumerate(pos):
    top, bot, left, right = var
   
    main_out[0, top:bot, left:right] = outs[i][0]
    if i == 7:
        break
# main_out[0, :512, 256:768] = outs[1][0]


In [ ]:
pos[8]

In [ ]:
main_out[0, 266:778, 0:512].shape

In [ ]:
plt.imshow(main_out.transpose(1,2,0)[:, :, 0])

In [ ]:
plt.imshow(ne_data[0].transpose(1,2,0)[:, :, -1])